In [7]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import re
import matplotlib.pyplot as plt
# 日本語フォントを使用する
plt.rcParams['font.family'] = 'IPAexGothic'

attractions = {
    0: 'ソアリン',
    1: 'トイストーリーマニア',
    2: 'タワーオブテラー',
    3: 'センターオブジアース',
    4: 'インディージョーンズ',
    5: 'レイジングスピリッツ',
    6: 'マジックランプシアター',
    7: 'タートルトーク'
}

In [8]:
file = "best_result_csv/best_results_230819_r100_t1500_review.csv"

In [9]:
# CSVファイルを読み込む
df = pd.read_csv(file)
display(df.head())
display(df.info())
display(df.shape)

,日付,トータル満足度,トータル所要時間,予定プラン,最適プラン,タイムスケジュール
0,2023-08-19,6085,672,"[5, 3, 2, 6, 5, 3, 5, 4, 3, 3, 6, 6, 4, 7, 4, 1]","[5, 3, 2, 6, 5, 3, 5, 4, 3, 6, 4, 7, 4]","[0, 30, 105, 165, 225, 270, 330, 390, 435, 495..."
1,2023-08-19,6194,672,"[2, 2, 4, 2, 4, 3, 4, 2, 2, 2, 4, 4, 7, 3, 4, ...","[2, 4, 2, 4, 3, 4, 2, 4, 7, 3, 4, 2, 4]","[0, 30, 75, 150, 210, 270, 330, 375, 435, 510,..."
2,2023-08-19,6166,671,"[4, 5, 2, 4, 3, 3, 2, 2, 4, 3, 2, 7, 4, 2, 4]","[4, 5, 2, 4, 3, 2, 4, 3, 2, 7, 4, 2, 4]","[0, 30, 60, 135, 195, 255, 300, 345, 420, 480,..."
3,2023-08-19,6174,673,"[2, 4, 3, 4, 7, 4, 2, 6, 2, 3, 4, 3, 4]","[2, 4, 3, 4, 7, 4, 2, 6, 2, 3, 4, 3, 4]","[0, 30, 75, 165, 210, 300, 330, 390, 450, 510,..."
4,2023-08-19,6177,673,"[2, 4, 3, 7, 3, 4, 3, 4, 4, 2, 2, 3, 2, 3, 4, 1]","[2, 4, 3, 7, 3, 4, 3, 4, 2, 3, 2, 3, 4]","[0, 30, 75, 165, 225, 300, 330, 390, 450, 510,..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   日付         100 non-null    object
 1   トータル満足度    100 non-null    int64 
 2   トータル所要時間   100 non-null    int64 
 3   予定プラン      100 non-null    object
 4   最適プラン      100 non-null    object
 5   タイムスケジュール  100 non-null    object
dtypes: int64(2), object(4)
memory usage: 4.8+ KB


None

(100, 6)

In [10]:
df_sorted = df.sort_values(by='トータル満足度', ascending=False)

In [11]:
display(df_sorted)

,日付,トータル満足度,トータル所要時間,予定プラン,最適プラン,タイムスケジュール
75,2023-08-19,6628,673,"[4, 5, 6, 2, 4, 2, 4, 3, 4, 2, 4, 3, 4, 4, 3]","[4, 5, 6, 2, 4, 2, 4, 3, 4, 2, 4, 3, 4, 3]","[0, 30, 60, 120, 165, 225, 285, 330, 390, 435,..."
9,2023-08-19,6627,674,"[4, 6, 2, 4, 2, 5, 4, 2, 4, 2, 3, 7, 4, 3, 3]","[4, 6, 2, 4, 2, 5, 4, 2, 4, 2, 3, 7, 4, 3]","[0, 30, 75, 135, 195, 270, 300, 345, 405, 465,..."
60,2023-08-19,6607,675,"[4, 6, 7, 2, 3, 2, 3, 3, 4, 4, 6, 4, 2, 3, 5, 4]","[4, 6, 7, 2, 3, 2, 3, 4, 6, 4, 2, 3, 5, 4]","[0, 30, 75, 150, 195, 255, 300, 360, 405, 465,..."
97,2023-08-19,6606,669,"[2, 5, 6, 2, 7, 4, 5, 4, 2, 7, 2, 4, 2, 2, 4]","[2, 5, 6, 2, 7, 4, 5, 4, 2, 7, 2, 4, 2, 4]","[0, 30, 60, 120, 165, 225, 285, 330, 375, 420,..."
80,2023-08-19,6602,672,"[4, 6, 3, 2, 3, 5, 4, 2, 6, 2, 4, 2, 4, 4, 3, ...","[4, 6, 3, 2, 3, 5, 4, 2, 6, 2, 4, 2, 4, 3]","[0, 30, 75, 150, 195, 255, 300, 345, 405, 450,..."
...,...,...,...,...,...,...
34,2023-08-19,5706,664,"[7, 4, 2, 4, 3, 4, 3, 2, 3, 3, 2, 4, 2, 0]","[7, 4, 2, 4, 3, 4, 3, 2, 3, 2, 4, 2]","[0, 60, 135, 195, 240, 315, 360, 420, 480, 540..."
10,2023-08-19,5700,670,"[2, 4, 3, 4, 3, 4, 0, 4, 6, 2, 4, 2, 1, 1, 1, 0]","[2, 4, 3, 4, 3, 4, 0, 4, 6, 2, 4, 2]","[0, 30, 75, 165, 210, 270, 330, 435, 480, 540,..."
72,2023-08-19,5700,670,"[4, 2, 6, 7, 0, 4, 7, 4, 2, 4, 6, 6, 4]","[4, 2, 6, 7, 0, 4, 7, 4, 2, 4, 6, 4]","[0, 30, 105, 165, 225, 315, 375, 435, 495, 555..."
42,2023-08-19,5699,658,"[7, 7, 2, 6, 6, 6, 7, 2, 4, 2, 7, 2, 4, 3, 4]","[7, 2, 6, 7, 2, 4, 2, 7, 2, 4, 3, 4]","[0, 60, 135, 180, 240, 315, 360, 405, 480, 525..."
